<a href="https://colab.research.google.com/github/Haj1h0/llm-from-scratch-code/blob/main/MultiHeadCausalSelfAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import math

class MultiHeadCausalSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads, context_length, dropout=0.0, qkv_bias=False):
        """
        d_model: 모델 차원 (임베딩 차원)
        num_heads: 헤드 수
        context_length: 최대 시퀀스 길이 (마스크 크기)
        dropout: 어텐션 가중치에 적용할 드롭아웃
        """
        super().__init__()
        assert d_model % num_heads == 0, "d_model은 num_heads로 나누어 떨어져야 합니다."
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        # Q, K, V를 한 번에 뽑는 선형층 (실무에서도 이 패턴 많이 씀)
        self.W_qkv = nn.Linear(d_model, 3 * d_model, bias=qkv_bias)

        # 여러 헤드 출력(concat) -> 다시 d_model로 투영하는 출력 projection
        self.W_out = nn.Linear(d_model, d_model)

        # 어텐션 가중치 드롭아웃
        self.attn_dropout = nn.Dropout(dropout
        # 출력 드롭아웃 (원하면 빼도 됨)
        self.proj_dropout = nn.Dropout(dropout)

        # 상삼각 마스크 (미래 토큰 가리기) - [context_length, context_length]
        mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
        self.register_buffer("mask", mask)  # 학습 파라미터는 아닌 버퍼로 등록

    def forward(self, x):
        """
        x: (batch, num_tokens, d_model)
        return: (batch, num_tokens, d_model)
        """
        b, n, d = x.shape
        assert d == self.d_model

        # 1) Q, K, V 한 번에 계산
        #    qkv: (b, n, 3 * d_model)
        qkv = self.W_qkv(x)
        # 2) q, k, v로 쪼개기
        #    각각 (b, n, d_model)
        q, k, v = qkv.chunk(3, dim=-1)

        # 3) 멀티헤드 형태로 reshape
        #    (b, n, num_heads, head_dim) -> (b, num_heads, n, head_dim)
        q = q.view(b, n, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(b, n, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(b, n, self.num_heads, self.head_dim).transpose(1, 2)
        # q, k, v: (b, h, n, d_head)

        # 4) 어텐션 스코어: Q K^T / sqrt(d_head)
        #    (b, h, n, d_head) @ (b, h, d_head, n) -> (b, h, n, n)
        attn_scores = q @ k.transpose(-2, -1)
        attn_scores = attn_scores / math.sqrt(self.head_dim)

        # 5) Causal 마스크 적용
        #    self.mask: (context_length, context_length)
        #    현재 시퀀스 길이 n만큼 잘라쓰기
        #    True인 위치에 -inf 채워서 softmax 후 0 되게 함
        causal_mask = self.mask[:n, :n].bool()  # (n, n)
        attn_scores = attn_scores.masked_fill(causal_mask, float('-inf'))

        # 6) 소프트맥스 + 드롭아웃
        attn_weights = torch.softmax(attn_scores, dim=-1)  # (b, h, n, n)
        attn_weights = self.attn_dropout(attn_weights)

        # 7) 문맥벡터 계산: (b, h, n, n) @ (b, h, n, d_head) -> (b, h, n, d_head)
        context = attn_weights @ v  # (b, h, n, d_head)

        # 8) 헤드 합치기: (b, h, n, d_head) -> (b, n, h * d_head = d_model)
        context = context.transpose(1, 2).contiguous()  # (b, n, h, d_head)
        context = context.view(b, n, self.d_model)      # (b, n, d_model)

        # 9) 최종 선형 변환 + 드롭아웃
        out = self.W_out(context)        # (b, n, d_model)
        out = self.proj_dropout(out)
        return out
